In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import zipfile
import os
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from scipy.sparse.linalg import svds
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import LatentDirichletAllocation
from sentence_transformers import SentenceTransformer
from numpy.linalg import pinv
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self-signed certificate in certificate chain
[nltk_data]     (_ssl.c:1000)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self-signed certificate in certificate chain
[nltk_data]     (_ssl.c:1000)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     self-signed certificate in certificate chain
[nltk_data]     (_ssl.c:1000)>


False

In [4]:
zip_file_path = '/home/student/vishaka/archive (5).zip'
extract_dir = 'reuters_data'

In [5]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

In [6]:
extracted_files = os.listdir(extract_dir)
print("Extracted files:", extracted_files)

Extracted files: ['ModApte_test.csv', 'ModApte_unused.csv', 'ModApte_train.csv', 'ModHayes_train.csv', 'ModHayes_test.csv', 'ModLewis_unused.csv', 'ModLewis_train.csv', 'ModLewis_test.csv']


In [13]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [19]:
import re
from nltk.corpus import stopwords
import spacy

# Load SpaCy's English model
nlp = spacy.load('en_core_web_sm')

# Define preprocessing functions
def clean_text(text):
    """Clean text by removing special characters, numbers, and extra spaces."""
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove digits
    text = text.lower()  # Convert to lowercase
    return text

def remove_stopwords(text):
    """Remove stopwords from the text."""
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def lemmatize_text_spacy(text):
    """Lemmatize text using SpaCy."""
    doc = nlp(text)  # Process text with SpaCy
    lemmatized_words = [token.lemma_ for token in doc if not token.is_punct]
    return ' '.join(lemmatized_words)

def preprocess_text_spacy(text):
    """Apply all preprocessing steps to the text using SpaCy for lemmatization."""
    text = clean_text(text)
    text = remove_stopwords(text)
    text = lemmatize_text_spacy(text)
    return text

ModHayes and ModLewis represent two different splitting strategies, each tailored for specific research purposes.ModLewis is the most widely used split, particularly for text classification benchmarks.ModHayes offers an alternative splitting strategy for experimentation.

In [15]:
train_df = pd.read_csv("/home/student/vishaka/reuters_data/ModLewis_train.csv")
test_df = pd.read_csv("/home/student/vishaka/reuters_data/ModLewis_test.csv")

In [16]:
print(train_df)

                                                    text text_type  \
0      Showers continued throughout the week in\nthe ...    "NORM"   
1      Standard Oil Co and BP North America\nInc said...    "NORM"   
2      Texas Commerce Bancshares Inc's Texas\nCommerc...    "NORM"   
3      BankAmerica Corp is not under\npressure to act...    "NORM"   
4      The U.S. Agriculture Department\nreported the ...    "NORM"   
...                                                  ...       ...   
13620  Australian trade unions said they have\nlaunch...    "NORM"   
13621  The ruling Liberal Democratic Party (LDP)\nhas...    "NORM"   
13622  Ministry of International Trade and\nIndustry ...    "NORM"   
13623  Sharp Corp will set up a 100 pct-owned\nsubsid...    "NORM"   
13624  A subsidiary of the Japanese\nconstruction gro...    "NORM"   

                                                  topics lewis_split  \
0                                              ['cocoa']     "TRAIN"   
1              

can  combine train and text while performing query retirval for efficient topics retrival on larger corpus

In [17]:
train_df['text'] = train_df['text'].astype(str)

In [21]:
train_df['cleaned_document'] = train_df['text'].apply(preprocess_text_spacy)

In [22]:
print(train_df.columns)

Index(['text', 'text_type', 'topics', 'lewis_split', 'cgis_split', 'old_id',
       'new_id', 'places', 'people', 'orgs', 'exchanges', 'date', 'title',
       'cleaned_document'],
      dtype='object')


In [24]:
documents = train_df['cleaned_document'].tolist()

In [25]:
print(documents[0])

shower continue throughout week bahia cocoa zone alleviate drought since early january improve prospect come temporao although normal humidity level restore comissaria smith say weekly review dry period mean temporao late year arrival week end february bag kilos make cumulative total season mln stage last year seem cocoa deliver early consignment include arrival figure comissaria smith say still doubt much old crop cocoa still available harvesting practically come end total bahia crop estimate around mln bag sale stand almost mln hundred thousand bag still hand farmer middleman exporter processor doubt much cocoa would fit export shipper experience dificultie obtaining bahia superior certificate view low quality recent week farmer sell good part cocoa hold consignment comissaria smith say spot bean price rise cruzado per arroba kilos bean shippers reluctant offer nearby shipment limited sale book march shipment dlrs per tonne port name new crop sale also light open port junejuly go dlr

In [27]:
sample=documents[:1000]
print(len(sample))

1000


In [29]:
def cur_decomposition(X, rank):
    """Perform CUR decomposition on a matrix X with a specified rank."""
    col_norms = np.sum(X**2, axis=0)
    row_norms = np.sum(X**2, axis=1)
    prob_cols = col_norms / np.sum(col_norms)
    prob_rows = row_norms / np.sum(row_norms)

    selected_cols = np.random.choice(X.shape[1], rank, replace=False, p=prob_cols)
    selected_rows = np.random.choice(X.shape[0], rank, replace=False, p=prob_rows)

    C = X[:, selected_cols]
    R = X[selected_rows, :]

    W = X[np.ix_(selected_rows, selected_cols)]
    U = np.linalg.pinv(W)

    return C, U, R

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def lda_gibbs_sampling_with_cur_tfidf(docs, num_topics, num_iter=1000, alpha=0.1, beta=0.1, rank=200, verbose=False):
    """
    Perform LDA topic modeling using CUR decomposition on a TF-IDF matrix.

    Args:
        docs: List of documents (strings).
        num_topics: Number of topics for LDA.
        num_iter: Number of Gibbs sampling iterations.
        alpha: Dirichlet prior for document-topic distribution.
        beta: Dirichlet prior for topic-word distribution.
        rank: Rank for CUR decomposition.
        verbose: Print progress during sampling.

    Returns:
        doc_topic_dist: Document-topic distribution matrix.
        topic_word_dist: Topic-word distribution matrix.
        vocab: Vocabulary list.
    """
    # Step 1: Create TF-IDF Matrix
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(docs).toarray()
    vocab = vectorizer.get_feature_names_out()  # Vocabulary list

    # Step 2: Apply CUR decomposition
    C, U, R = cur_decomposition(tfidf_matrix, rank)
    reduced_matrix = C@U@R

    # Step 3: Use reduced_matrix for LDA
    num_docs, vocab_size = reduced_matrix.shape
    topic_assignments = np.random.randint(0, num_topics, size=(num_docs, vocab_size))

    doc_topic_counts = np.zeros((num_docs, num_topics))
    topic_word_counts = np.zeros((num_topics, vocab_size))
    topic_counts = np.zeros(num_topics)

    for d in range(num_docs):
        for w in range(vocab_size):
            topic = topic_assignments[d, w]
            doc_topic_counts[d, topic] += reduced_matrix[d, w]
            topic_word_counts[topic, w] += reduced_matrix[d, w]
            topic_counts[topic] += reduced_matrix[d, w]

    for iteration in range(num_iter):
        if verbose:
            print(f"Iteration {iteration + 1}/{num_iter}")

        for d in range(num_docs):
            if verbose and d % 10 == 0:  # Print progress for every 10 documents
                print(f"Processing document {d + 1}/{num_docs}")

            for w in range(vocab_size):
                word_count = reduced_matrix[d, w]
                if word_count == 0:
                    continue

                topic = topic_assignments[d, w]
                doc_topic_counts[d, topic] -= word_count
                topic_word_counts[topic, w] -= word_count
                topic_counts[topic] -= word_count

                topic_probs = (
                    (doc_topic_counts[d, :] + alpha)
                    * (topic_word_counts[:, w] + beta)
                    / (topic_counts + beta * vocab_size)
                )
                topic_probs = np.maximum(topic_probs, 0)  # Ensure non-negative
                prob_sum = topic_probs.sum()
                if prob_sum > 0:
                    topic_probs /= prob_sum
                else:
                    topic_probs = np.ones(num_topics) / num_topics

                new_topic = np.random.choice(num_topics, p=topic_probs)
                topic_assignments[d, w] = new_topic
                doc_topic_counts[d, new_topic] += word_count
                topic_word_counts[new_topic, w] += word_count
                topic_counts[new_topic] += word_count

    doc_topic_dist = (doc_topic_counts + alpha) / (doc_topic_counts.sum(axis=1, keepdims=True) + num_topics * alpha)
    topic_word_dist = (topic_word_counts + beta) / (topic_word_counts.sum(axis=1, keepdims=True) + vocab_size * beta)

    return doc_topic_dist, topic_word_dist, vocab

In [31]:

def dynamic_query_expansion(query, documents, model, top_n=3, original_weight=0.7, expanded_weight=0.3):
    unique_terms = set(' '.join(documents).split())
    term_embeddings = {term: model.encode(term) for term in unique_terms}

    query_terms = query.split()
    query_embeddings = [model.encode(term) for term in query_terms]

    query_embedding = np.mean(query_embeddings, axis=0)
    similarities = {
        term: cosine_similarity([query_embedding], [embedding])[0][0]
        for term, embedding in term_embeddings.items()
    }
    expanded_terms = sorted(similarities, key=similarities.get, reverse=True)[:top_n]

    expanded_embeddings = [term_embeddings[term] for term in expanded_terms]
    combined_embedding = (
        original_weight * np.mean(query_embeddings, axis=0) +
        expanded_weight * np.mean(expanded_embeddings, axis=0)
    )

    expanded_query_terms = set(query_terms).union(expanded_terms)
    return combined_embedding, ' '.join(expanded_query_terms)

In [32]:
def retrieve_documents_with_cur(query, documents, model, lda_topics, lda_vocab, rank=200, top_n=5):
    query_embedding, expanded_query = dynamic_query_expansion(query, documents, model)
    print(f"Expanded Query: '{expanded_query}'\n")
    document_embeddings = model.encode(documents)

    print("Applying CUR Decomposition to reduce document embeddings dimensionality...")
    C, U, R = cur_decomposition(document_embeddings, rank=rank)
    reduced_document_embeddings = C @ U
    #reduced_query_embedding = query_embedding @ (U@R).T
    reduced_query_embedding = query_embedding @ (R).T


    similarities = cosine_similarity([reduced_query_embedding], reduced_document_embeddings).flatten()
    top_indices = np.argsort(-similarities)[:top_n]

    print("LDA Topics Distribution for Top Documents:")
    for idx in top_indices:
        doc_topics = lda_topics[idx]
        print(f"Document: {documents[idx]}\nTopic Distribution: {doc_topics}\n")

    return top_indices

In [35]:
import os
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

# Set the proxy environment variables
os.environ['HTTP_PROXY'] = 'socks5h://127.0.0.1:1080'
os.environ['HTTPS_PROXY'] = 'socks5h://127.0.0.1:1080'

# Explicitly download the model
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Wrap it with SentenceTransformer for compatibility
sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("Model loaded successfully.")


Model loaded successfully.


In [36]:
lda_topics, lda_word_dist, vocab = lda_gibbs_sampling_with_cur_tfidf(sample, num_topics=5, num_iter=1000,verbose=True)

Iteration 1/1000
Processing document 1/1000
Processing document 11/1000
Processing document 21/1000
Processing document 31/1000
Processing document 41/1000
Processing document 51/1000
Processing document 61/1000
Processing document 71/1000
Processing document 81/1000
Processing document 91/1000
Processing document 101/1000
Processing document 111/1000
Processing document 121/1000
Processing document 131/1000
Processing document 141/1000
Processing document 151/1000
Processing document 161/1000
Processing document 171/1000
Processing document 181/1000
Processing document 191/1000
Processing document 201/1000
Processing document 211/1000
Processing document 221/1000
Processing document 231/1000
Processing document 241/1000
Processing document 251/1000
Processing document 261/1000
Processing document 271/1000
Processing document 281/1000
Processing document 291/1000
Processing document 301/1000
Processing document 311/1000
Processing document 321/1000
Processing document 331/1000
Processi

KeyboardInterrupt: 

In [ ]:
print('LDA Topics')
print(lda_topics)
print('LDA Word Distribution')
print(lda_word_dist)
print('Vocab')
print(vocab)

LDA Topics


NameError: name 'lda_topics' is not defined

In [ ]:
import pickle
lda_model_1000 = {
    'lda_topics': lda_topics,
    'lda_word_dist': lda_word_dist,
    'vocab': vocab
}

with open('lda_model_1000.pkl', 'wb') as file:
    pickle.dump(lda_model_1000, file)

with open('lda_model_1000.pkl', 'rb') as file:
    loaded_lda_model = pickle.load(file)

# Access the components
lda_topics = loaded_lda_model['lda_topics']
lda_word_dist = loaded_lda_model['lda_word_dist']
vocab = loaded_lda_model['vocab']

# Verify the contents
print("Loaded Document-Topic Distribution Shape:",lda_topics.shape)
print("Loaded Topic-Word Distribution Shape:",lda_word_dist.shape)
print("Loaded Vocabulary Size:", len(vocab))

In [ ]:
print(len(vocab))

In [ ]:
print(len(lda_word_dist[0]))

In [ ]:
query = "bahia cocoa zone"
print("\nRetrieving Documents with CUR Decomposition and Query Expansion...")
top_indices = retrieve_documents_with_cur(query,sample, model, lda_topics, vocab, rank=4)


In [ ]:
print("\nTop Matching Documents:")
for idx in top_indices:
    print(f"Document: {documents[idx]}")


Evaluation

In [ ]:
tokenized_documents = [doc.split() for doc in sample]
for i in tokenized_documents:
  print(i)

In [ ]:
print(type(sample))

In [ ]:
example=train_df.head(1000)

In [ ]:
vocab1 = {word: idx for idx, word in enumerate(vocab)}
def tokenize_document(text, vocab):
    text = str(text)
    return [vocab[word] for word in text.split() if word in vocab]
tokenized_documents = example[
    'cleaned_text'
].apply(lambda x: tokenize_document(x, vocab1)).tolist()

print("Vocabulary:", vocab)
print("Tokenized Documents:", tokenized_documents)

In [ ]:
print(len(vocab))

Word Cloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

# Assuming LDA Word Distribution and Vocab are given as:
lda_word_distribution = lda_word_dist

# Number of topics
num_topics = len(lda_word_distribution)

for topic_idx in range(num_topics):
    # Get the word probabilities for this topic
    word_probs = lda_word_distribution[topic_idx]

    # Create a dictionary with word -> probability
    word_freq = {vocab[i]: word_probs[i] for i in range(len(vocab))}

    # Create word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

    # Plot the word cloud for the current topic
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Topic {topic_idx + 1}')
    plt.show()


Perplexity

In [ ]:
import math
def compute_perplexity(lda_word_dist, lda_topic_dist, tokenized_documents):
    total_log_prob = 0
    total_words = 0

    for doc_idx, doc in enumerate(tokenized_documents):
        for word_idx in doc:
            # Probability of the word given the document
            word_prob = np.dot(lda_topic_dist[doc_idx], lda_word_dist[:, word_idx])
            if word_prob > 0:
                total_log_prob += math.log(word_prob)
            total_words += 1

    # Compute perplexity
    perplexity = math.exp(-total_log_prob / total_words)
    return perplexity

In [ ]:
perplexity = compute_perplexity(lda_word_dist.T, lda_topics.T, tokenized_documents)
print("Perplexity:", perplexity)

Coherence score

In [ ]:
from collections import Counter

def compute_coherence(lda_word_dist, vocab, tokenized_documents, top_n=4):
    num_topics, vocab_size = lda_word_dist.shape
    coherence_scores = []

    # Get word co-occurrences from documents
    word_count = Counter()
    for doc in tokenized_documents:
        word_count.update(doc)

    # Compute coherence for each topic
    for topic_idx in range(num_topics):
        # Get top N words for the topic
        top_word_indices = lda_word_dist[topic_idx].argsort()[-top_n:][::-1]
        top_words = [vocab[i] for i in top_word_indices]

        # Calculate pairwise coherence for top words
        coherence = 0
        for i, word1 in enumerate(top_words):
            for j, word2 in enumerate(top_words):
                if i < j:
                    word1_idx, word2_idx = vocab.index(word1), vocab.index(word2)
                    # Co-occurrence approximation
                    word1_count = word_count[word1_idx]
                    word2_count = word_count[word2_idx]
                    pair_count = sum(1 for doc in tokenized_documents if word1_idx in doc and word2_idx in doc)

                    # Compute log coherence score
                    if pair_count > 0:
                        coherence += math.log((pair_count + 1) / word2_count)

        coherence_scores.append(coherence)

    # Average coherence score over topics
    avg_coherence = np.mean(coherence_scores)
    return avg_coherence

coherence_score = compute_coherence(lda_word_dist, vocab, tokenized_documents)
print("Coherence Score:", coherence_score)


In [ ]:
def retrieve_documents(query, documents, lda_topics, lda_word_dist, vocab, model, top_n=5):
    expanded_embedding, expanded_query = dynamic_query_expansion(query, documents, model)
    print(f"Original Query: '{query}'")
    print(f"Expanded Query: '{expanded_query}'\n")

    query_topic_dist = infer_query_topic_distribution(expanded_query, lda_word_dist, vocab)
    print(f"Expanded Query Topic Distribution: {query_topic_dist}\n")

    similarities_lda = cosine_similarity([query_topic_dist], lda_topics).flatten()
    top_indices_lda = np.argsort(-similarities_lda)[:top_n]

    C, U, R = cur_decomposition(document_embeddings, rank=4)
    reduced_document_embeddings = C @ U
    #reduced_query_embedding = query_embedding @ (U@R).T
    reduced_query_embedding = expanded_embedding @ (R).T


    similarities_direct = cosine_similarity([reduced_query_embedding], reduced_document_embeddings).flatten()
    top_indices_direct = np.argsort(-similarities_direct)[:top_n]

    print("\nTop Matching Documents (Expanded Query with LDA):")
    for idx in top_indices_lda:
        print(f"Document: {documents[idx]}\nTopic Similarity: {similarities_lda[idx]}\n")

    print("\nTop Matching Documents (Expanded Query with Direct Embedding Similarity):")
    for idx in top_indices_direct:
        print(f"Document: {documents[idx]}\nSimilarity: {similarities_direct[idx]}\n")

    return top_indices_lda, top_indices_direct

In [ ]:
query = "bahia cocoa zone"
print("\nRetrieving Documents with Expanded Query and LDA...")
top_indices_lda, top_indices_direct = retrieve_documents(
query, documents, lda_topics, lda_word_dist, vocab, model
)

print("\nComparison of Results:")
print(f"LDA-based Retrieval Top Documents: {[documents[idx] for idx in top_indices_lda]}")
print(f"Direct Embedding-based Retrieval Top Documents: {[documents[idx] for idx in top_indices_direct]}")

In [ ]:
def compute_perplexity(word_dist, vocab, corpus):
    """
    Computes the perplexity for the LDA model.
    word_dist: Word distribution from LDA (topics x vocab size)
    vocab: Vocabulary list of words (list of strings)
    corpus: List of documents, where each document is a list of word indices
    """
    total_log_likelihood = 0
    total_words = 0

    for doc in corpus:
        doc_likelihood = 0
        for word_idx in doc:
            # Get topic probabilities for the word (word_dist is topics x vocab size)
            word_probs = word_dist[:, word_idx]  # Get topic distribution for the word

            # Add log-probability of the word given the topics
            doc_likelihood += np.log(np.sum(word_probs))  # Sum over topics

        total_log_likelihood += doc_likelihood
        total_words += len(doc)

    # Perplexity formula
    perplexity = np.exp(-total_log_likelihood / total_words)
    return perplexity

# Example usage:
# 'lda_word_dist' is your word distribution matrix (topics x vocab size)
# 'vocab' is the list of words (should be in the same order as the indices in lda_word_dist)
# 'sample' is the corpus, where each document is a list of word indices
perplexity = compute_perplexity(word_dist=lda_word_dist, vocab=vocab, corpus=tokenized_documents)
print("Perplexity:", perplexity)


In [ ]:
def compute_coherence_score(word_dist, vocab, top_n=10):
    """
    Computes the coherence score for the LDA model.
    word_dist: Word distribution from LDA (topics x vocab size)
    vocab: Vocabulary list of words
    top_n: Number of top words to consider for coherence score calculation
    """
    # Extract top N words for each topic
    top_words = []
    for topic_idx in range(word_dist.shape[0]):
        # Get the top N words in the topic based on their probability
        top_indices = np.argsort(word_dist[topic_idx])[::-1][:top_n]
        top_words.append([vocab[i] for i in top_indices])

    # Compute coherence score based on word pairs' similarity
    vectorizer = CountVectorizer(stop_words='english')
    total_score = 0
    total_pairs = 0

    for words in top_words:
        # Compute pairwise similarity between the words
        word_pairs = [(words[i], words[j]) for i in range(len(words)) for j in range(i + 1, len(words))]
        total_pairs += len(word_pairs)

        # Vectorize the words to compute cosine similarity
        vecs = vectorizer.fit_transform([' '.join(words[i:i+2]) for i in range(len(words)-1)])
        cosine_sim = cosine_similarity(vecs)

        # Add pairwise coherence score
        total_score += np.sum(cosine_sim)

    # Average coherence score
    coherence_score = total_score / total_pairs if total_pairs > 0 else 0
    return coherence_score

# Example usage
# LDA_word_distribution: your LDA model word distribution (topics x vocab size)
coherence_score = compute_coherence_score(word_dist=LDA_word_distribution, vocab=vocab, top_n=10)
print("Coherence Score:", coherence_score)


In [ ]:
def evaluate_retrieval(query, retrieved_indices, true_topic, documents, lda_topics):
    retrieved_topics = [lda_topics[idx] for idx in retrieved_indices]
    relevant_indices = [i for i, doc_topics in enumerate(lda_topics) if true_topic in doc_topics]

    true_positives = len(set(retrieved_indices) & set(relevant_indices))
    precision = true_positives / len(retrieved_indices)
    recall = true_positives / len(relevant_indices)
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1_score

query = "bahia cocoa zone"
true_topic = "cocoa"
#retrieved_indices = retrieve_documents_with_cur(query, sample, model, lda_topics, vocab, rank=2)

precision, recall, f1_score = evaluate_retrieval(query, top_indices, true_topic, sample, lda_topics)
print(f"Precision: {precision:.2f}, Recall: {recall:.2f}, F1 Score: {f1_score:.2f}")


In [ ]:
vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, stop_words='english')
A = vectorizer.fit_transform(documents)
vocabulary = vectorizer.get_feature_names_out()
print("Term-Document Matrix Shape:", A.shape)

In [ ]:
k = 100
U, Sigma, VT = svds(A, k=k)
row_leverage_scores = np.sum(U**2, axis=1)
column_leverage_scores = np.sum(VT.T**2, axis=1)

In [ ]:
num_samples = 50
sampled_rows = np.random.choice(A.shape[0], size=num_samples, replace=False, p=row_leverage_scores / np.sum(row_leverage_scores))
sampled_columns = np.random.choice(A.shape[1], size=num_samples, replace=False, p=column_leverage_scores / np.sum(column_leverage_scores))

In [ ]:
C = A[:, sampled_columns]
R = A[sampled_rows, :]
U_matrix = np.dot(np.linalg.pinv(C.todense()), A.todense()).dot(np.linalg.pinv(R.todense()))

In [ ]:
print("C Shape:", C.shape)
print("R Shape:", R.shape)
print("U Matrix Shape:", U_matrix.shape)

In [ ]:
selected_columns = np.random.choice(R.shape[1], size=13625, replace=False)
R_reduced = R[:, selected_columns]

In [ ]:
C_expanded = np.zeros((17815, 50))
C_expanded[:13625, :] = C.toarray()

In [ ]:
print(R.shape)

In [ ]:
print(C_expanded.shape)

noraml retrival process just for viewing results

In [ ]:
query = "AUSTRALIAN UNIONS  NEW SOUTH WALES STRIKES "
query_cleaned = preprocess_text(query)
#print(query_cleaned)
query_vector = vectorizer.transform([query_cleaned])
#print(query_vector)

R = R.toarray() if hasattr(R, 'toarray') else np.array(R)
if R.ndim == 1:
    R = R.reshape(1, -1)
query_vector = query_vector.toarray() if hasattr(query_vector, 'toarray') else np.array(query_vector)
if query_vector.ndim == 1:
    query_vector = query_vector.reshape(1, -1)
R_pinv = np.linalg.pinv(R)
query_reduced = np.dot(query_vector, R_pinv)
similarities = cosine_similarity(query_reduced, C)

top_n = 5
top_n_indices = np.argsort(similarities[0])[-top_n:][::-1]
retrieved_docs = [documents[i] for i in top_n_indices]

print("Query:", query)
print("Top Retrieved Documents:")
for i, doc in enumerate(retrieved_docs, start=1):
    print(f"{i}: {doc[:200]}...")

In [ ]:
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(A)
num_top_words = 10
for idx, topic in enumerate(lda.components_):
    top_terms = [vocabulary[i] for i in topic.argsort()[-num_top_words:][::-1]]
    print(f"Topic {idx + 1}: {', '.join(top_terms)}")

In [ ]:
print(C[0][0])

Using C and passing it to lda model ,transforming query using R and passing it to lda model

In [ ]:
lda = LatentDirichletAllocation(n_components=10, random_state=42)
lda.fit(C_expanded.T)

In [ ]:
query = "AUSTRALIAN UNIONS  NEW SOUTH WALES STRIKES "
query_cleaned = preprocess_text(query)
#print(query_cleaned)
query_vector = vectorizer.transform([query_cleaned])
print(query_vector)
print(query_vector.shape)

R = R.toarray() if hasattr(R, 'toarray') else np.array(R)
if R.ndim == 1:
    R = R.reshape(1, -1)
query_vector = query_vector.toarray() if hasattr(query_vector, 'toarray') else np.array(query_vector)
if query_vector.ndim == 1:
    query_vector = query_vector.reshape(1, -1)
R_pinv = np.linalg.pinv(R)
query_reduced = np.dot(query_vector, R_pinv)


In [ ]:
print(query_reduced.shape)

In [ ]:
query_vector = vectorizer.transform([query])  # Has same dimensions as the training data (13625 features)
query_topic_distribution = lda.transform(query_vector)

In [ ]:
query_topic_distribution = lda.transform(query_reduced)

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# CUR Decomposition
def cur_decomposition(X, rank):
    """
    Perform CUR decomposition on a matrix X with a specified rank.
    """
    col_norms = np.sum(X**2, axis=0)
    row_norms = np.sum(X**2, axis=1)
    prob_cols = col_norms / np.sum(col_norms)
    prob_rows = row_norms / np.sum(row_norms)

    selected_cols = np.random.choice(X.shape[1], rank, replace=False, p=prob_cols)
    selected_rows = np.random.choice(X.shape[0], rank, replace=False, p=prob_rows)

    C = X[:, selected_cols]
    R = X[selected_rows, :]

    W = X[np.ix_(selected_rows, selected_cols)]
    U = np.linalg.pinv(W)

    return C, U, R

# LDA Topic Modeling using Gibbs Sampling
def lda_gibbs_sampling(docs, num_topics, num_iter=1000, alpha=0.1, beta=0.1):
    vocab = list(set(word for doc in docs for word in doc.split()))
    term_doc_matrix = np.zeros((len(docs), len(vocab)))

    for i, doc in enumerate(docs):
        for word in doc.split():
            term_doc_matrix[i, vocab.index(word)] += 1

    num_docs, vocab_size = term_doc_matrix.shape
    topic_assignments = np.random.randint(0, num_topics, size=(num_docs, vocab_size))

    doc_topic_counts = np.zeros((num_docs, num_topics))
    topic_word_counts = np.zeros((num_topics, vocab_size))
    topic_counts = np.zeros(num_topics)

    for d in range(num_docs):
        for w in range(vocab_size):
            topic = topic_assignments[d, w]
            doc_topic_counts[d, topic] += term_doc_matrix[d, w]
            topic_word_counts[topic, w] += term_doc_matrix[d, w]
            topic_counts[topic] += term_doc_matrix[d, w]

    for _ in range(num_iter):
        for d in range(num_docs):
            for w in range(vocab_size):
                word_count = term_doc_matrix[d, w]
                if word_count == 0:
                    continue

                topic = topic_assignments[d, w]
                doc_topic_counts[d, topic] -= word_count
                topic_word_counts[topic, w] -= word_count
                topic_counts[topic] -= word_count

                topic_probs = (
                    (doc_topic_counts[d, :] + alpha)
                    * (topic_word_counts[:, w] + beta)
                    / (topic_counts + beta * vocab_size)
                )
                topic_probs /= topic_probs.sum()

                new_topic = np.random.choice(num_topics, p=topic_probs)
                topic_assignments[d, w] = new_topic
                doc_topic_counts[d, new_topic] += word_count
                topic_word_counts[new_topic, w] += word_count
                topic_counts[new_topic] += word_count

    doc_topic_dist = (doc_topic_counts + alpha) / (doc_topic_counts.sum(axis=1, keepdims=True) + num_topics * alpha)
    topic_word_dist = (topic_word_counts + beta) / (topic_word_counts.sum(axis=1, keepdims=True) + vocab_size * beta)
    return doc_topic_dist, topic_word_dist, vocab

# Dynamic Query Expansion with Weighting
def dynamic_query_expansion(query, documents, model, top_n=3, original_weight=0.7, expanded_weight=0.3):
    unique_terms = set(' '.join(documents).split())
    term_embeddings = {term: model.encode(term) for term in unique_terms}

    query_terms = query.split()
    query_embeddings = [model.encode(term) for term in query_terms]

    query_embedding = np.mean(query_embeddings, axis=0)
    similarities = {
        term: cosine_similarity([query_embedding], [embedding])[0][0]
        for term, embedding in term_embeddings.items()
    }
    expanded_terms = sorted(similarities, key=similarities.get, reverse=True)[:top_n]

    expanded_embeddings = [term_embeddings[term] for term in expanded_terms]
    combined_embedding = (
        original_weight * np.mean(query_embeddings, axis=0) +
        expanded_weight * np.mean(expanded_embeddings, axis=0)
    )

    expanded_query_terms = set(query_terms).union(expanded_terms)
    return combined_embedding, ' '.join(expanded_query_terms)

# Retrieve Documents
def retrieve_documents(query, documents, document_embeddings, model, lda_topics, lda_vocab, top_n=5):
    query_embedding, expanded_query = dynamic_query_expansion(query, documents, model)
    print(f"Original Query: '{query}'")
    print(f"Expanded Query: '{expanded_query}'")

    similarities = cosine_similarity([query_embedding], document_embeddings).flatten()
    top_indices = np.argsort(-similarities)[:top_n]

    print("\nLDA Topics Distribution for Top Documents:")
    for idx in top_indices:
        doc_topics = lda_topics[idx]
        print(f"Document: {documents[idx]}\nTopic Distribution: {doc_topics}\n")

    return top_indices

# Example Usage
if __name__ == "__main__":
    documents = [
        "The sky is blue and beautiful.",
        "Love this blue and bright sky!",
        "The quick brown fox jumps over the lazy dog.",
        "A king's breakfast has sausages, ham, and bacon.",
        "I love green eggs, ham, sausages, and bacon!",
        "The brown fox is quick and the blue dog is lazy!",
        "The sky is very blue and the sky is very beautiful today.",
        "The dog is lazy but the brown fox is quick!"
    ]

    model = SentenceTransformer('all-MiniLM-L6-v2')
    document_embeddings = model.encode(documents)

    print("Performing CUR Decomposition...")
    C, U, R = cur_decomposition(document_embeddings, rank=4)
    print(f"CUR Decomposition Results: C: {C.shape}, U: {U.shape}, R: {R.shape}")

    print("\nPerforming LDA Topic Modeling...")
    lda_topics, lda_word_dist, vocab = lda_gibbs_sampling(documents, num_topics=3, num_iter=1000)

    query = "blue sky"
    print("\nRetrieving Documents...")
    top_indices = retrieve_documents(query, documents, document_embeddings, model, lda_topics, vocab)

    print("\nTop Matching Documents:")
    for idx in top_indices:
        print(f"Document: {documents[idx]}")

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA

# CUR Decomposition
def cur_decomposition(X, rank):
    """Perform CUR decomposition on a matrix X with a specified rank."""
    col_norms = np.sum(X**2, axis=0)
    row_norms = np.sum(X**2, axis=1)
    prob_cols = col_norms / np.sum(col_norms)
    prob_rows = row_norms / np.sum(row_norms)

    selected_cols = np.random.choice(X.shape[1], rank, replace=False, p=prob_cols)
    selected_rows = np.random.choice(X.shape[0], rank, replace=False, p=prob_rows)

    C = X[:, selected_cols]
    R = X[selected_rows, :]

    W = X[np.ix_(selected_rows, selected_cols)]
    U = np.linalg.pinv(W)

    return C, U, R

# LDA Topic Modeling using Gibbs Sampling
def lda_gibbs_sampling(docs, num_topics, num_iter=1000, alpha=0.1, beta=0.1):
    vocab = list(set(word for doc in docs for word in doc.split()))
    term_doc_matrix = np.zeros((len(docs), len(vocab)))

    for i, doc in enumerate(docs):
        for word in doc.split():
            term_doc_matrix[i, vocab.index(word)] += 1

    num_docs, vocab_size = term_doc_matrix.shape
    topic_assignments = np.random.randint(0, num_topics, size=(num_docs, vocab_size))

    doc_topic_counts = np.zeros((num_docs, num_topics))
    topic_word_counts = np.zeros((num_topics, vocab_size))
    topic_counts = np.zeros(num_topics)

    for d in range(num_docs):
        for w in range(vocab_size):
            topic = topic_assignments[d, w]
            doc_topic_counts[d, topic] += term_doc_matrix[d, w]
            topic_word_counts[topic, w] += term_doc_matrix[d, w]
            topic_counts[topic] += term_doc_matrix[d, w]

    for _ in range(num_iter):
        for d in range(num_docs):
            for w in range(vocab_size):
                word_count = term_doc_matrix[d, w]
                if word_count == 0:
                    continue

                topic = topic_assignments[d, w]
                doc_topic_counts[d, topic] -= word_count
                topic_word_counts[topic, w] -= word_count
                topic_counts[topic] -= word_count

                topic_probs = (
                    (doc_topic_counts[d, :] + alpha)
                    * (topic_word_counts[:, w] + beta)
                    / (topic_counts + beta * vocab_size)
                )
                topic_probs /= topic_probs.sum()

                new_topic = np.random.choice(num_topics, p=topic_probs)
                topic_assignments[d, w] = new_topic
                doc_topic_counts[d, new_topic] += word_count
                topic_word_counts[new_topic, w] += word_count
                topic_counts[new_topic] += word_count

    doc_topic_dist = (doc_topic_counts + alpha) / (doc_topic_counts.sum(axis=1, keepdims=True) + num_topics * alpha)
    topic_word_dist = (topic_word_counts + beta) / (topic_word_counts.sum(axis=1, keepdims=True) + vocab_size * beta)

    return doc_topic_dist, topic_word_dist, vocab

# Retrieve Documents (Including CUR Reduction)
def retrieve_documents_with_cur(query, documents, model, lda_topics, lda_vocab, rank=10, top_n=5):
    # Step 1: Generate embeddings for documents
    document_embeddings = model.encode(documents)

    # Step 2: Apply CUR Decomposition
    print("Applying CUR Decomposition to reduce document embeddings dimensionality...")
    C, U, R = cur_decomposition(document_embeddings, rank=rank)
    reduced_document_embeddings = C @ U

    # Step 3: Reduce the query embedding to the same dimensionality (using PCA)
    query_embedding = model.encode(query)

    # Fit PCA to the document embeddings and use it to transform the query embedding
    pca = PCA(n_components=rank)
    pca.fit(reduced_document_embeddings)  # Fit on the reduced document embeddings
    reduced_query_embedding = pca.transform([query_embedding])[0]  # Project query into reduced space

    # Step 4: Calculate cosine similarity between query and reduced embeddings
    similarities = cosine_similarity([reduced_query_embedding], reduced_document_embeddings).flatten()
    top_indices = np.argsort(-similarities)[:top_n]

    # Step 5: Output the results and topic distribution for each document
    print(f"Original Query: '{query}'\n")
    print("LDA Topics Distribution for Top Documents:")
    for idx in top_indices:
        doc_topics = lda_topics[idx]
        print(f"Document: {documents[idx]}\nTopic Distribution: {doc_topics}\n")

    return top_indices

# Example Usage
if __name__ == "__main__":
    documents = [
        "The sky is blue and beautiful.",
        "Love this blue and bright sky!",
        "The quick brown fox jumps over the lazy dog.",
        "A king's breakfast has sausages, ham, and bacon.",
        "I love green eggs, ham, sausages, and bacon!",
        "The brown fox is quick and the blue dog is lazy!",
        "The sky is very blue and the sky is very beautiful today.",
        "The dog is lazy but the brown fox is quick!"
    ]

    model = SentenceTransformer('all-MiniLM-L6-v2')

    print("\nPerforming LDA Topic Modeling...")
    lda_topics, lda_word_dist, vocab = lda_gibbs_sampling(documents, num_topics=3, num_iter=1000)

    query = "blue sky"
    print("\nRetrieving Documents with CUR Decomposition...")
    top_indices = retrieve_documents_with_cur(query, documents, model, lda_topics, vocab, rank=4)

    print("\nTop Matching Documents:")
    for idx in top_indices:
        print(f"Document: {documents[idx]}")


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# CUR Decomposition
def cur_decomposition(X, rank):
    """Perform CUR decomposition on a matrix X with a specified rank."""
    col_norms = np.sum(X**2, axis=0)
    row_norms = np.sum(X**2, axis=1)
    prob_cols = col_norms / np.sum(col_norms)
    prob_rows = row_norms / np.sum(row_norms)

    selected_cols = np.random.choice(X.shape[1], rank, replace=False, p=prob_cols)
    selected_rows = np.random.choice(X.shape[0], rank, replace=False, p=prob_rows)

    C = X[:, selected_cols]
    R = X[selected_rows, :]

    W = X[np.ix_(selected_rows, selected_cols)]
    U = np.linalg.pinv(W)

    return C, U, R

# LDA Topic Modeling using Gibbs Sampling
def lda_gibbs_sampling(docs, num_topics, num_iter=1000, alpha=0.1, beta=0.1):
    vocab = list(set(word for doc in docs for word in doc.split()))
    term_doc_matrix = np.zeros((len(docs), len(vocab)))

    for i, doc in enumerate(docs):
        for word in doc.split():
            term_doc_matrix[i, vocab.index(word)] += 1

    num_docs, vocab_size = term_doc_matrix.shape
    topic_assignments = np.random.randint(0, num_topics, size=(num_docs, vocab_size))

    doc_topic_counts = np.zeros((num_docs, num_topics))
    topic_word_counts = np.zeros((num_topics, vocab_size))
    topic_counts = np.zeros(num_topics)

    for d in range(num_docs):
        for w in range(vocab_size):
            topic = topic_assignments[d, w]
            doc_topic_counts[d, topic] += term_doc_matrix[d, w]
            topic_word_counts[topic, w] += term_doc_matrix[d, w]
            topic_counts[topic] += term_doc_matrix[d, w]

    for _ in range(num_iter):
        for d in range(num_docs):
            for w in range(vocab_size):
                word_count = term_doc_matrix[d, w]
                if word_count == 0:
                    continue

                topic = topic_assignments[d, w]
                doc_topic_counts[d, topic] -= word_count
                topic_word_counts[topic, w] -= word_count
                topic_counts[topic] -= word_count

                topic_probs = (
                    (doc_topic_counts[d, :] + alpha)
                    * (topic_word_counts[:, w] + beta)
                    / (topic_counts + beta * vocab_size)
                )
                topic_probs /= topic_probs.sum()

                new_topic = np.random.choice(num_topics, p=topic_probs)
                topic_assignments[d, w] = new_topic
                doc_topic_counts[d, new_topic] += word_count
                topic_word_counts[new_topic, w] += word_count
                topic_counts[new_topic] += word_count

    doc_topic_dist = (doc_topic_counts + alpha) / (doc_topic_counts.sum(axis=1, keepdims=True) + num_topics * alpha)
    topic_word_dist = (topic_word_counts + beta) / (topic_word_counts.sum(axis=1, keepdims=True) + vocab_size * beta)

    return doc_topic_dist, topic_word_dist, vocab

# Retrieve Documents (Including CUR Reduction)
def retrieve_documents_with_cur(query, documents, model, lda_topics, lda_vocab, rank=10, top_n=5):
    # Step 1: Generate embeddings for documents
    document_embeddings = model.encode(documents)

    # Step 2: Apply CUR Decomposition
    print("Applying CUR Decomposition to reduce document embeddings dimensionality...")
    C, U, R = cur_decomposition(document_embeddings, rank=rank)
    reduced_document_embeddings = C @ U

    # Step 3: Project the query embedding into the same reduced space (using U from CUR)
    query_embedding = model.encode(query)
    print(query_embedding.shape)
    print(R.shape)
    reduced_query_embedding = query_embedding.T @ R.T

    # The query embedding should be projected into the same space as the reduced document embeddings using U
    #reduced_query_embedding = query_embedding.T @ U.T  # Project query into the reduced space

    # Step 4: Calculate cosine similarity between query and reduced embeddings
    similarities = cosine_similarity([reduced_query_embedding], reduced_document_embeddings).flatten()
    top_indices = np.argsort(-similarities)[:top_n]

    # Step 5: Output the results and topic distribution for each document
    print(f"Original Query: '{query}'\n")
    print("LDA Topics Distribution for Top Documents:")
    for idx in top_indices:
        doc_topics = lda_topics[idx]
        print(f"Document: {documents[idx]}\nTopic Distribution: {doc_topics}\n")

    return top_indices

# Example Usage
if __name__ == "__main__":
    documents = [
        "The sky is blue and beautiful.",
        "Love this blue and bright sky!",
        "The quick brown fox jumps over the lazy dog.",
        "A king's breakfast has sausages, ham, and bacon.",
        "I love green eggs, ham, sausages, and bacon!",
        "The brown fox is quick and the blue dog is lazy!",
        "The sky is very blue and the sky is very beautiful today.",
        "The dog is lazy but the brown fox is quick!"
    ]

    model = SentenceTransformer('all-MiniLM-L6-v2')

    print("\nPerforming LDA Topic Modeling...")
    lda_topics, lda_word_dist, vocab = lda_gibbs_sampling(documents, num_topics=3, num_iter=1000)

    query = "blue sky"
    print("\nRetrieving Documents with CUR Decomposition...")
    top_indices = retrieve_documents_with_cur(query, documents, model, lda_topics, vocab, rank=4)

    print("\nTop Matching Documents:")
    for idx in top_indices:
        print(f"Document: {documents[idx]}")


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer


def cur_decomposition(X, rank):
    """Perform CUR decomposition on a matrix X with a specified rank."""
    col_norms = np.sum(X**2, axis=0)
    row_norms = np.sum(X**2, axis=1)
    prob_cols = col_norms / np.sum(col_norms)
    prob_rows = row_norms / np.sum(row_norms)

    selected_cols = np.random.choice(X.shape[1], rank, replace=False, p=prob_cols)
    selected_rows = np.random.choice(X.shape[0], rank, replace=False, p=prob_rows)

    C = X[:, selected_cols]
    R = X[selected_rows, :]

    W = X[np.ix_(selected_rows, selected_cols)]
    U = np.linalg.pinv(W)

    return C, U, R

def lda_gibbs_sampling(docs, num_topics, num_iter=1000, alpha=0.1, beta=0.1):
    vocab = list(set(word for doc in docs for word in doc.split()))
    term_doc_matrix = np.zeros((len(docs), len(vocab)))

    for i, doc in enumerate(docs):
        for word in doc.split():
            term_doc_matrix[i, vocab.index(word)] += 1

    num_docs, vocab_size = term_doc_matrix.shape
    topic_assignments = np.random.randint(0, num_topics, size=(num_docs, vocab_size))

    doc_topic_counts = np.zeros((num_docs, num_topics))
    topic_word_counts = np.zeros((num_topics, vocab_size))
    topic_counts = np.zeros(num_topics)

    for d in range(num_docs):
        for w in range(vocab_size):
            topic = topic_assignments[d, w]
            doc_topic_counts[d, topic] += term_doc_matrix[d, w]
            topic_word_counts[topic, w] += term_doc_matrix[d, w]
            topic_counts[topic] += term_doc_matrix[d, w]

    for _ in range(num_iter):
        for d in range(num_docs):
            for w in range(vocab_size):
                word_count = term_doc_matrix[d, w]
                if word_count == 0:
                    continue

                topic = topic_assignments[d, w]
                doc_topic_counts[d, topic] -= word_count
                topic_word_counts[topic, w] -= word_count
                topic_counts[topic] -= word_count

                topic_probs = (
                    (doc_topic_counts[d, :] + alpha)
                    * (topic_word_counts[:, w] + beta)
                    / (topic_counts + beta * vocab_size)
                )
                topic_probs /= topic_probs.sum()

                new_topic = np.random.choice(num_topics, p=topic_probs)
                topic_assignments[d, w] = new_topic
                doc_topic_counts[d, new_topic] += word_count
                topic_word_counts[new_topic, w] += word_count
                topic_counts[new_topic] += word_count

    doc_topic_dist = (doc_topic_counts + alpha) / (doc_topic_counts.sum(axis=1, keepdims=True) + num_topics * alpha)
    topic_word_dist = (topic_word_counts + beta) / (topic_word_counts.sum(axis=1, keepdims=True) + vocab_size * beta)

    return doc_topic_dist, topic_word_dist, vocab


def dynamic_query_expansion(query, documents, model, top_n=3, original_weight=0.7, expanded_weight=0.3):
    unique_terms = set(' '.join(documents).split())
    term_embeddings = {term: model.encode(term) for term in unique_terms}

    query_terms = query.split()
    query_embeddings = [model.encode(term) for term in query_terms]

    query_embedding = np.mean(query_embeddings, axis=0)
    similarities = {
        term: cosine_similarity([query_embedding], [embedding])[0][0]
        for term, embedding in term_embeddings.items()
    }
    expanded_terms = sorted(similarities, key=similarities.get, reverse=True)[:top_n]

    expanded_embeddings = [term_embeddings[term] for term in expanded_terms]
    combined_embedding = (
        original_weight * np.mean(query_embeddings, axis=0) +
        expanded_weight * np.mean(expanded_embeddings, axis=0)
    )

    expanded_query_terms = set(query_terms).union(expanded_terms)
    return combined_embedding, ' '.join(expanded_query_terms)

def retrieve_documents_with_cur(query, documents, model, lda_topics, lda_vocab, rank=10, top_n=5):
    query_embedding, expanded_query = dynamic_query_expansion(query, documents, model)
    print(f"Expanded Query: '{expanded_query}'\n")
    document_embeddings = model.encode(documents)

    print("Applying CUR Decomposition to reduce document embeddings dimensionality...")
    C, U, R = cur_decomposition(document_embeddings, rank=rank)
    reduced_document_embeddings = C @ U
    reduced_query_embedding = query_embedding @ R.T


    similarities = cosine_similarity([reduced_query_embedding], reduced_document_embeddings).flatten()
    top_indices = np.argsort(-similarities)[:top_n]

    print("LDA Topics Distribution for Top Documents:")
    for idx in top_indices:
        doc_topics = lda_topics[idx]
        print(f"Document: {documents[idx]}\nTopic Distribution: {doc_topics}\n")

    return top_indices
if __name__ == "__main__":
    documents = [
        "The sky is blue and beautiful.",
        "Love this blue and bright sky!",
        "The quick brown fox jumps over the lazy dog.",
        "A king's breakfast has sausages, ham, and bacon.",
        "I love green eggs, ham, sausages, and bacon!",
        "The brown fox is quick and the blue dog is lazy!",
        "The sky is very blue and the sky is very beautiful today.",
        "The dog is lazy but the brown fox is quick!"
    ]

    model = SentenceTransformer('all-MiniLM-L6-v2')

    print("\nPerforming LDA Topic Modeling...")
    lda_topics, lda_word_dist, vocab = lda_gibbs_sampling(documents, num_topics=3, num_iter=1000)

    query = "blue sky"
    print("\nRetrieving Documents with CUR Decomposition and Query Expansion...")
    top_indices = retrieve_documents_with_cur(query, documents, model, lda_topics, vocab, rank=4)

    print("\nTop Matching Documents:")
    for idx in top_indices:
        print(f"Document: {documents[idx]}")
